In [47]:
import math
import numpy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import sys
import joblib
import warnings
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import RFE
from mlxtend.feature_selection import SequentialFeatureSelector as Sfs
from sklearn.feature_selection import chi2
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from dateutil.relativedelta import relativedelta
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error
from collections import Counter
import plotly.express as px
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [48]:
sns.set()

In [49]:
# Data Import and overview
DATASET_PATH = "../../data/tweets_2020_2021_v2.csv"
df = pd.read_csv(filepath_or_buffer=DATASET_PATH, sep=",")
df = df.sort_values(by='timestamp', ascending=True)
df.tail()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,tweet_id,text,timestamp,user_id,like_count,retweet_count,...,day_phase,week_idx,day_phase_enc,day_of_week_enc,month_enc,year_enc,sentiment_enc,verified_enc,seniority,topics_cleaned
612261,612261,612261,612261,612261,1477036834261770241,…. them crawfish was so good yall lmfao,2021-12-31 21:59:57+00:00,795707439219884037,0,0,...,Night,2021-52,4,0,2,1,2,0,5,NaN
98252,98252,98252,98252,98252,1477036835603836932,Damn all the freaks are at the New Bev right n...,2021-12-31 21:59:58+00:00,34679503,26,0,...,Night,2021-52,4,0,2,1,1,0,13,NaN
1072384,1072384,1072384,1072384,1072384,1477036838217035777,Fast And Professional Vehicle Lockout Service ...,2021-12-31 21:59:58+00:00,210241982,0,0,...,Night,2021-52,4,0,2,1,0,0,11,NaN
1072385,1072385,1072385,1072385,1072385,1477036840024567809,Very much this https://t.co/b3bgtstLfr,2021-12-31 21:59:59+00:00,883576549,3,0,...,Night,2021-52,4,0,2,1,1,0,9,NaN
1072386,1072386,1072386,1072386,1072386,1477036841933185025,Best thread ever https://t.co/EBo4JkTCx4,2021-12-31 21:59:59+00:00,131371694,0,0,...,Night,2021-52,4,0,2,1,2,0,12,NaN


In [110]:
vars = ['like_count', 'retweet_count', 'quote_count', 'reply_count', 'reach', 'topics_ids', 'sentiment_enc', 'day_phase_enc', 'day_of_week_enc', 'month_enc', 'popularity', 'followers', 'following', 'tweet_count', 'verified_enc', 'seniority']
tweet_vars = ['like_count', 'retweet_count', 'quote_count', 'reply_count', 'reach', 'sentiment_enc', 'day_phase_enc', 'day_of_week_enc', 'month_enc', 'topics_ids']
users_vars = ['followers', 'following', 'tweet_count', 'verified_enc', 'seniority']
num_vars = ['like_count', 'retweet_count', 'quote_count', 'reply_count', 'reach', 'followers', 'following', 'tweet_count', 'seniority']
cat_vars = ['sentiment_enc', 'verified_enc', 'day_of_week_enc', 'day_phase_enc', 'month_enc']
variables_to_keep = ['like_count', 'reply_count', 'reach', 'followers', 'month_enc', 'verified_enc', 'day_phase_enc', 'proc_time', 'retweet_count']

In [51]:
df_2020 = df[df['year'] == 2020]
df_2021 = df[df['year'] == 2021]

In [52]:
df_2020.shape

(534942, 35)

In [53]:
df_2021.shape

(537445, 35)

# Machine Learning

### Predicting tweet popularity in 2021 using 2020 data with topics

## Data spliting

In [54]:
num_folds = 7
seed = 7
scoring = 'accuracy'
validation_size = 0.70

### Only data from selected topic

In [95]:
df_2020_topic = df_2020[(~df_2020['topics'].isnull()) & (df_2020['topics_cleaned'] == 'Person')]
#df_2020_topic = df_2020_topic[:100]
df_2020_topic.shape

(48349, 35)

In [96]:
df_2021_topic = df_2021[(~df_2021['topics'].isnull()) & (df_2021['topics_cleaned'] == 'Person')]
#df_2021_topic = df_2021_topic[:100]
df_2021_topic.shape

(31801, 35)

In [97]:
df_2020_topic['timestamp'] = pd.to_datetime(df_2020_topic['timestamp'])
df_2020_topic['proc_time'] = [i.replace(tzinfo=datetime.timezone.utc) for i in df_2020_topic['timestamp']]
df_2020_topic

/var/folders/7r/g2lhjv3x7bb_lgcnhj7_q9hr0000gn/T/ipykernel_1067/4105467309.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/7r/g2lhjv3x7bb_lgcnhj7_q9hr0000gn/T/ipykernel_1067/4105467309.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,tweet_id,text,timestamp,user_id,like_count,retweet_count,...,week_idx,day_phase_enc,day_of_week_enc,month_enc,year_enc,sentiment_enc,verified_enc,seniority,topics_cleaned,proc_time
35,35,35,35,35,1212176423189909506,and he still cheating https://t.co/zYAQ8wc6Vg,2020-01-01 00:59:15,3322006549,2,1,...,2020-01,1,6,4,0,0,0,6,Person,2020-01-01 00:59:15+00:00
41,41,41,41,41,1212176426088181760,Disgruntled Pope Francis pulls himself free fr...,2020-01-01 00:59:15,858762062,0,0,...,2020-01,1,6,4,0,2,0,9,Person,2020-01-01 00:59:15+00:00
92,92,92,92,92,1212176433457569799,Rihanna will be named the Petty Queen of the D...,2020-01-01 00:59:17,95088762,0,0,...,2020-01,1,6,4,0,2,0,12,Person,2020-01-01 00:59:17+00:00
103,103,103,103,103,1212176432794722304,#phish stream locked and loaded. No FOMO here....,2020-01-01 00:59:17,153245307,0,0,...,2020-01,1,6,4,0,0,0,11,Person,2020-01-01 00:59:17+00:00
1753,1753,1753,1753,1753,1212176455507034112,I’m 5’ 10” very hot 🤣 https://t.co/PhR4kLUeJz,2020-01-01 00:59:22,281482400,0,0,...,2020-01,1,6,4,0,2,0,11,Person,2020-01-01 00:59:22+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522751,522751,522751,522751,522751,1344765232649003011,"A black cop supporting trump, bro. Find the ne...",2020-12-31 21:59:50+00:00,913269161073299456,0,0,...,2020-53,4,4,2,0,2,0,4,Person,2020-12-31 21:59:50+00:00
68748,68748,68748,68748,68748,1344765245307326465,This judge speaks the truth. Donald Trump is a...,2020-12-31 21:59:53+00:00,4745216682,0,0,...,2020-53,4,4,2,0,0,0,6,Person,2020-12-31 21:59:53+00:00
737164,737164,737164,737164,737164,1344765245701697536,I ain’t gonna hold y’all but Tory Lanez have s...,2020-12-31 21:59:53+00:00,3072562841,0,0,...,2020-53,4,4,2,0,0,0,7,Person,2020-12-31 21:59:53+00:00
505196,505196,505196,505196,505196,1344765243201880066,Yes this vote via Electoral College must b rev...,2020-12-31 21:59:53+00:00,899380600506920961,0,0,...,2020-53,4,4,2,0,2,0,4,Person,2020-12-31 21:59:53+00:00


In [111]:
topic_performance_2020 = df_2020_topic[variables_to_keep].resample('D', on='proc_time').mean()
topic_performance_2020

,like_count,reply_count,reach,followers,month_enc,verified_enc,day_phase_enc,retweet_count
proc_time,,,,,,,,
2020-01-01 00:00:00+00:00,50.026316,1.513158,5888.105263,3592.684211,4.0,0.039474,1.486842,9.539474
2020-01-02 00:00:00+00:00,3.098901,0.318681,5963.934066,5823.604396,4.0,0.032967,2.010989,0.483516
2020-01-03 00:00:00+00:00,4.536000,0.696000,9264.904000,8985.808000,4.0,0.040000,2.016000,0.528000
2020-01-04 00:00:00+00:00,5.108696,0.413043,4946.467391,3480.369565,4.0,0.032609,1.652174,0.760870
2020-01-05 00:00:00+00:00,2.804348,0.195652,3024.217391,3007.260870,4.0,0.010870,2.086957,0.858696
...,...,...,...,...,...,...,...,...
2020-12-27 00:00:00+00:00,2.121622,0.216216,12038.162162,11939.162162,2.0,0.040541,2.013514,0.162162
2020-12-28 00:00:00+00:00,36.297872,1.765957,18040.797872,3527.244681,2.0,0.042553,2.021277,1.393617
2020-12-29 00:00:00+00:00,9.181102,0.417323,6991.606299,6804.795276,2.0,0.062992,2.070866,1.031496


In [112]:
df_2021_topic['timestamp'] = pd.to_datetime(df_2021_topic['timestamp'])
df_2021_topic['proc_time'] = [i.replace(tzinfo=datetime.timezone.utc) for i in df_2021_topic['timestamp']]
#df_2021_topic = df_2021_topic.set_index(df.timestamp)

/var/folders/7r/g2lhjv3x7bb_lgcnhj7_q9hr0000gn/T/ipykernel_1067/2796613702.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/7r/g2lhjv3x7bb_lgcnhj7_q9hr0000gn/T/ipykernel_1067/2796613702.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [113]:
topic_performance_2021 = df_2021_topic[variables_to_keep].resample('D', on='proc_time').mean()
topic_performance_2021

,like_count,reply_count,reach,followers,month_enc,verified_enc,day_phase_enc,retweet_count
proc_time,,,,,,,,
2021-01-01 00:00:00+00:00,3.384615,0.384615,3621.707692,3395.384615,4.0,0.046154,1.984615,0.476923
2021-01-02 00:00:00+00:00,3.548387,0.387097,2305.182796,2043.580645,4.0,0.043011,1.827957,1.043011
2021-01-03 00:00:00+00:00,2.880000,0.464000,2805.648000,2076.752000,4.0,0.016000,2.328000,0.400000
2021-01-04 00:00:00+00:00,31.873950,1.428571,15234.991597,5803.210084,4.0,0.033613,1.983193,8.436975
2021-01-05 00:00:00+00:00,2.253968,0.468254,2760.126984,2514.198413,4.0,0.015873,2.277778,0.365079
...,...,...,...,...,...,...,...,...
2021-12-27 00:00:00+00:00,2.634921,0.301587,2847.301587,2842.238095,2.0,0.047619,2.079365,0.253968
2021-12-28 00:00:00+00:00,3.671233,0.301370,4335.082192,3839.534247,2.0,0.041096,2.054795,0.561644
2021-12-29 00:00:00+00:00,8.849315,0.424658,3592.821918,3352.465753,2.0,0.054795,2.219178,0.602740


In [142]:
X_train = topic_performance_2020.drop('retweet_count', axis=1)
y_train = topic_performance_2020['retweet_count']
print(X_train.shape)
print(y_train.shape)

(366, 7)
(366,)


In [143]:
X_test = topic_performance_2021.drop('retweet_count', axis=1)
y_test = topic_performance_2021['retweet_count']
print(X_test.shape)
print(y_test.shape)

(365, 7)
(365,)


In [144]:
def train_time_series_with_folds(X_train, X_test, y_train, y_test):
    #create, train and do inference of the model
    model = GradientBoostingRegressor(random_state=42)
    model.fit(X_train, y_train)

    print("model trained")

    predictions = model.predict(X_test)

    print("model done")

    #calculate MAE
    mae = np.round(mean_absolute_error(y_test, predictions), 3)

    res = []
    for i in range(365):
        d = dict()
        d['timestamp'] = topic_performance_2021.index[i]
        d['type'] = 'real'
        d['value'] = y_test.iloc[i]
        res.append(d)

        d = dict()
        d['timestamp'] = topic_performance_2021.index[i]
        d['type'] = 'prediction'
        d['value'] = predictions[i]
        res.append(d)

    df_results = pd.DataFrame(res)

    fig = px.line(df_results, x="timestamp", y="value", color='type', title=("Predictions of topic retweet count average for 2021 with MAE:" + str(mae)),
             color_discrete_sequence=px.colors.qualitative.Safe, width=900, height=500)
    #fig.add_trace()
    fig.update_xaxes(title_text="Time")
    fig.update_yaxes(title_text="Retweet count")
    fig.show()


    #create a dataframe with the variable importances of the model
    df_importances = pd.DataFrame({
        'feature': model.feature_names_in_,
        'importance': model.feature_importances_
    }).sort_values(by='importance', ascending=False)

    fig = px.bar(df_importances, x="feature", y="importance", title=("Variable Importances"),
             color_discrete_sequence=px.colors.qualitative.Safe, width=900, height=500)
    fig.show()

In [145]:
train_time_series_with_folds(X_train, X_test, y_train, y_test)

model trained
model done
